In [2]:
import sqlite3
from datetime import datetime

#creating database and connecting

conn=sqlite3.connect("attendance.db")
cursor=conn.cursor()

#creating 2 tables

cursor.execute("""create table if not exists
employee_info(emp_id integer primary key autoincrement,
name text not null, 
department text)""")

cursor.execute("""create table if not exists
attendance_log(log_id integer primary key autoincrement,
emp_id int,
checkin datetime,
checkout datetime,
foreign key (emp_id) references employee_info(emp_id))""")

conn.commit()
print("tables created successfully")

tables created successfully


In [3]:
#adding values to employees

def add_employee():
    count=int(input("enter the no. of employees:"))
    for i in range(count):
        
        e_name=input("enter the name:")
        e_dept=input("enter the dept:")

    cursor.execute("insert into employee_info (name,department) values (?,?)",(e_name,e_dept))

    conn.commit()
    print("values inserted successfully")

add_employee()

enter the no. of employees: 2
enter the name: sangee
enter the dept: HR
enter the name: sindhu
enter the dept: Banking


values inserted successfully


In [4]:
#mark attendance - check in

def mark_checkin():
    emp_id=int(input("enter the employee id for checkin time:"))
    checkin_time=datetime.now()

    cursor.execute("""insert into attendance_log(emp_id, checkin) values(?,?)""", (emp_id, checkin_time))
    conn.commit()

    print(f"checkin recorded as {checkin_time} for emp_Id {emp_id}")

mark_checkin()

enter the employee id for checkin time: 1


checkin recorded as 2025-07-13 06:50:21.883657 for emp_Id 1


In [9]:
#mark attendance - check out

def mark_checkout():
    emp_id=int(input("enter the employee id for checkout time:"))
    checkout_time=datetime.now()

    cursor.execute(""" insert into attendance_log(emp_id,checkout) values(?,?)""",(emp_id,checkout_time))
    conn.commit()

    print(f"checkout recorded as {checkout_time} for emp_Id {emp_id}")

mark_checkout()



#bt we should not give like this  - its incorrect
#both checkin and checkout should come in same row - bt here it will come in different rows
#bcoz insert alwz create new row in table -- even if we give values for 1 or 2 columns
#insert doesnt find existing row --alwz creates new one 

#so it will be in one by one next line
#bt we want in same line nearby checkin 
#so we hv to check the checkin of emp. and find the recent null(empty) space in checkout -
#there time should be filled - so check the below code to perform it

#so we should use update - it edits and existing row

enter the employee id for checkout time: 1


checkout recorded as 2025-07-12 19:14:34.244329 for emp_Id 1


In [5]:
def mark_checkout():
    emp_id=int(input("enter the employee id for checkout time:"))
    checkout_time=datetime.now()

    cursor.execute(""" update attendance_log set checkout=? where emp_id=? and checkout is null""",(checkout_time,emp_id))
    conn.commit()

    if cursor.rowcount>0:
        print(f"checkout recorded as {checkout_time} for emp_Id {emp_id}")
    else:
        print("no open checkin found for this employee. Maybe already checked out or invalid ID")

mark_checkout()


enter the employee id for checkout time: 1


checkout recorded as 2025-07-13 06:50:39.993819 for emp_Id 1


In [6]:
#to view in table form

import pandas as pd

def view_attendance_table():
    query="select * from attendance_log order by checkin desc"
    df=pd.read_sql_query(query,conn)

    if df.empty:
        print("no attendance records found")

    else:
        print("attendace log")

        display(df)

view_attendance_table()

attendace log


,log_id,emp_id,checkin,checkout
0,4,1,2025-07-13 06:50:21.883657,2025-07-13 06:50:39.993819
1,2,2,2025-07-12 18:51:48.263251,None
2,1,2,2025-07-12 18:51:05.230586,None
3,3,1,None,2025-07-12 19:14:34.244329


In [7]:
#will try one by one

add_employee()

enter the no. of employees: 5
enter the name: anbazagan
enter the dept: EB
enter the name: saraswathi
enter the dept: homemaker
enter the name: sangeetha
enter the dept: IT
enter the name: sindhu
enter the dept: Banking
enter the name: vijaya
enter the dept: homemaker


values inserted successfully


In [8]:
mark_checkin()

enter the employee id for checkin time: 3


checkin recorded as 2025-07-13 07:15:23.849452 for emp_Id 3


In [17]:
#will write code for checkout with validation 

def mark_checkout():
    emp_id=int(input("enter the emp_id for checkout:"))
    checkout_time=datetime.now()    #stores current date time

    cursor.execute("""select log_id from attendance_log where emp_id-? and
                    checkin is not null and checkout is null
                    order by log_id desc
                    limit 1""",(emp_id,))

    result=cursor.fetchone()   #gets first row from result , if nothing matches returns null

    if result:
        log_id=result[0]

    cursor.execute("""update attendance_log set checkout=? where log_id=?""",(checkout_time, log_id))

    conn.commit()



mark_checkout()



enter the emp_id for checkout: 3


attendace log


,log_id,emp_id,checkin,checkout
0,5,3,2025-07-13 07:15:23.849452,None
1,4,1,2025-07-13 06:50:21.883657,2025-07-13 06:50:39.993819
2,2,2,2025-07-12 18:51:48.263251,2025-07-13 07:29:29.565601
3,1,2,2025-07-12 18:51:05.230586,2025-07-13 07:29:49.588985
4,3,1,None,2025-07-12 19:14:34.244329


In [16]:
view_attendance_table()

attendace log


,log_id,emp_id,checkin,checkout
0,5,3,2025-07-13 07:15:23.849452,None
1,4,1,2025-07-13 06:50:21.883657,2025-07-13 06:50:39.993819
2,2,2,2025-07-12 18:51:48.263251,2025-07-13 07:29:29.565601
3,1,2,2025-07-12 18:51:05.230586,None
4,3,1,None,2025-07-12 19:14:34.244329


In [25]:
from datetime import datetime  # To get current date & time

def mark_checkout():
    try:
        # Ask for employee ID
        emp_id = int(input("Enter the Employee ID for checkout: "))

        # Capture the current time to use as checkout time
        checkout_time = datetime.now()

        # Find the latest log where:
        # - the employee has already checked in
        # - but not yet checked out (checkout is NULL)
        cursor.execute("""
            SELECT log_id FROM attendance_log
            WHERE emp_id = ? AND checkin IS NOT NULL AND checkout IS NULL
            ORDER BY log_id DESC
            LIMIT 1
        """, (emp_id,))

        result = cursor.fetchone()  # Fetch one result: e.g., (7,)

        # If a matching open check-in exists
        if result:
            log_id = result[0]  # Extract the log ID from the tuple

            # Update that row by setting the checkout time
            cursor.execute("""
                UPDATE attendance_log
                SET checkout = ?
                WHERE log_id = ?
            """, (checkout_time, log_id))

            conn.commit()  # Save the change to the database

            print(f"✅ Checkout recorded at {checkout_time} for Employee ID {emp_id}")
        
        else:
            # No open check-in found for this employee
            print("⚠️ No active check-in found. Please check in first.")

    except ValueError:
        # If the user enters a non-integer (like text instead of a number)
        print("❌ Invalid input. Please enter a valid numeric Employee ID.")
    
    except Exception as e:
        # Any other unexpected error
        print(f"❌ Error: {e}")

add_employee()

mark_checkin()

mark_checkout()

view_attendance_table()

enter the no. of employees: 2
enter the name: san
enter the dept: it
enter the name: sid
enter the dept: bank


values inserted successfully


enter the employee id for checkin time: 2


checkin recorded as 2025-07-13 08:00:26.969730 for emp_Id 2


Enter the Employee ID for checkout:  2


✅ Checkout recorded at 2025-07-13 08:00:28.332348 for Employee ID 2
attendace log


,log_id,emp_id,checkin,checkout
0,9,2,2025-07-13 08:00:26.969730,2025-07-13 08:00:28.332348
1,8,2,2025-07-13 07:36:08.955743,2025-07-13 07:36:10.945528
2,7,1,2025-07-13 07:33:48.938051,2025-07-13 07:33:53.388631
3,6,4,2025-07-13 07:33:05.157430,2025-07-13 07:33:09.086117
4,5,3,2025-07-13 07:15:23.849452,None
5,4,1,2025-07-13 06:50:21.883657,2025-07-13 06:50:39.993819
6,2,2,2025-07-12 18:51:48.263251,2025-07-13 07:29:29.565601
7,1,2,2025-07-12 18:51:05.230586,2025-07-13 07:29:49.588985
8,3,1,None,2025-07-12 19:14:34.244329


In [27]:
#to view attendance with employee info -- join with sql

import pandas as pd

def view_attendance_with_details():
    query=""" select
        al.log_id,
        ei.emp_id,
        ei.name,
        ei.department,
        al.checkin,
        al.checkout
        from attendance_log al join employee_info ei
        on al.emp_id=ei.emp_id
        order by al.checkin desc"""

    df=pd.read_sql_query(query, conn)

    if df.empty:
        print("no attendance records found")

    else:
        print("attendance log with employee info:")

        display(df)

view_attendance_with_details()

attendance log with employee info:


,log_id,emp_id,name,department,checkin,checkout
0,9,2,sindhu,Banking,2025-07-13 08:00:26.969730,2025-07-13 08:00:28.332348
1,8,2,sindhu,Banking,2025-07-13 07:36:08.955743,2025-07-13 07:36:10.945528
2,7,1,sang,HR,2025-07-13 07:33:48.938051,2025-07-13 07:33:53.388631
3,6,4,vijaya,homemaker,2025-07-13 07:33:05.157430,2025-07-13 07:33:09.086117
4,5,3,sindhu,Banking,2025-07-13 07:15:23.849452,None
5,4,1,sang,HR,2025-07-13 06:50:21.883657,2025-07-13 06:50:39.993819
6,2,2,sindhu,Banking,2025-07-12 18:51:48.263251,2025-07-13 07:29:29.565601
7,1,2,sindhu,Banking,2025-07-12 18:51:05.230586,2025-07-13 07:29:49.588985
8,3,1,sang,HR,None,2025-07-12 19:14:34.244329


In [28]:
#add total working hours

cursor.execute("""alter table attendance_log add column duration real""")
conn.commit()

print("duration column added")

duration column added


In [30]:
#calculate and update duration on checkout

from datetime import datetime

def update_work_duration():    #fetch all records where checkout is done bt duration is still null

    cursor.execute("""select log_id, checkin, checkout from attendance_log
                    where checkout is not null and duration is null""")
    
    rows=cursor.fetchall()

    for log_id, checkin_str, checkout_str in rows:
        checkin=datetime.fromisoformat(checkin_str)
        checkout=datetime.fromisoformat(checkout_str)
        duration=(checkout-checkin).total_seconds() / 3600    #in hours

        cursor.execute("""update attendance_log set duration=? where log_id=?""",(duration,log_id))  #update duration in DB
        conn.commit()
        print("work duration updated for all completed logs")

update_work_duration()
              
    #error bcoz it is not in string -- so chec below code


work duration updated for all completed logs
work duration updated for all completed logs


TypeError: fromisoformat: argument must be str

In [33]:
#calculate and update duration 

from datetime import datetime

def update_work_duration():    #fetch all records where checkout is done bt duration is still null

    cursor.execute("""select log_id, checkin, checkout from attendance_log
                    where checkout is not null and duration is null""")
    
    rows=cursor.fetchall()

    for log_id, checkin_str, checkout_str in rows:  #if values are already datetime skip this - if string code as below

        if isinstance(checkin_str,str):
            checkin=datetime.fromisoformat(checkin_str)

        else:
            checkin=checkin_str

        if isinstance(checkout_str,str):
            checkout=datetime.fromisoformat(checkout_str)

        else:
            checkout=checkout_str
            
        duration=(checkout - checkin).total_seconds() / 3600    #in hours

        cursor.execute("""update attendance_log set duration=? where log_id=?""",(duration,log_id))  #update duration in DB
        conn.commit()
        print("work duration updated for all completed logs")

update_work_duration()

#either checkout or checkin is none -- so check below validation code

TypeError: unsupported operand type(s) for -: 'datetime.datetime' and 'NoneType'

In [35]:
#calculate and update duration 

from datetime import datetime

def update_work_duration():    #fetch all records where checkout is done bt duration is still null

    cursor.execute("""select log_id, checkin, checkout from attendance_log
                    where checkout is not null and duration is null""")
    
    rows=cursor.fetchall()

    for log_id, checkin_str, checkout_str in rows:  #if values are already datetime skip this - if string code as below

        if checkin_str is None or checkout_str is None:
            continue
            
        if isinstance(checkin_str,str):
            checkin=datetime.fromisoformat(checkin_str)

        else:
            checkin=checkin_str

        if isinstance(checkout_str,str):
            checkout=datetime.fromisoformat(checkout_str)

        else:
            checkout=checkout_str
            
        duration=(checkout - checkin).total_seconds() / 3600    #in hours

        cursor.execute("""update attendance_log set duration=? where log_id=?""",(duration,log_id))  #update duration in DB
        conn.commit()
    print("work duration updated for all completed logs")

update_work_duration()

work duration updated for all completed logs


In [38]:

#list currently working employees

def currently_working():
    query="""select
        ei.emp_id,
        ei.name,
        ei.department,
        al.checkin,
        al.checkout 
        from attendance_log al join employee_info ei
        on al.emp_id = ei.emp_id
        where al.checkout is null"""

    df=pd.read_sql_query(query,conn)

    if df.empty:
        print("no employees are currently working")

    else:
        print("employees currently present:")

        display(df)

currently_working()

employees currently present:


,emp_id,name,department,checkin,checkout
0,3,sindhu,Banking,2025-07-13 07:15:23.849452,None


In [42]:
#to prevent double checkin-- few lines of code should be written inside checkin function

def mark_checkin():
    emp_id=int(input("enter the employee id for checkin time:"))
    checkin_time=datetime.now()

#prevent double checkin -- should be before inserting new checkin

    cursor.execute("""select * from attendance_log where emp_id=? and checkout is null
                    order by checkin desc limit 1""", (emp_id,))

    result=cursor.fetchone()

    if result:
        print(f"employee id {emp_id} has already checked in at {result[2]} and not checked out yet")
        return    #exit function if already checked in

    #proceed to insert checkin

    cursor.execute("""insert into attendance_log(emp_id, checkin) values(?,?)""", (emp_id, checkin_time))
    conn.commit()

    print(f"checkin recorded as {checkin_time} for emp_Id {emp_id}")

mark_checkin()


enter the employee id for checkin time: 1


employee id 1 has already checkd in at 2025-07-13 09:46:25.335929 and not checked out yet
